## 计算图

### 三种计算图的概念

有三种计算图的构建方式：

- 静态计算：在TensorFlow1.0时代，采用的是静态计算图
- 动态计算图：在TensorFlow2.0时代，采用的是动态计算图
- Autograph

优缺点：

- 动态计算图即Eager Excution的好处是方便调试程序
    - 写起来就像写numpy一样，所见即所得；
    - 方便实现各种日志打印；
    - 控制流。
- 使用动态计算图的缺点是运行效率相对会低一些
    - 动态图会有许多次Python进程和TensorFlow的C++进程之间的通信

如何在TensorFlow2.0中使用静态图:
- 可以使用@tf.function装饰器将普通Python函数转换成对应的TensorFlow计算图构建代码。
- 运行该函数就相当于在TensorFlow1.0中用Session执行代码。


###  计算图的组成


计算图主要由：
- 节点(nodes)
    - 表示操作符Operator
    - 称之为算子
- 线(edges)
    - 表示计算间的依赖
    - 实线表示有数据传递依赖：传递的数据即张量
    - 虚线通常可以表示控制依赖：即执行先后顺序

![](imgs/01.jpg)


### 静态计算图

在TensorFlow1.0中，使用静态计算图分两步：
- 第一步定义计算图，
- 第二步在会话中执行计算图。

具体案例如下：

In [1]:
# 在tensorflow 1.中的计算图实现
import tensorflow as tf

g = tf.Graph()

# 第一步： 定义计算图
with g.as_default():
    #placeholder为占位符，执行会话时候指定填充对象
    x = tf.placeholder(name="x",shape=[],dtype=tf.string)
    y = tf.placeholder(name="y",shape=[],dtype=tf.string)
    z = tf.string_join([x,y],name="join",separator=" ")

# 第二步在会话中执行计算图
with tf.Session(graph=g) as sess:
    print(tf.__version__)
    print(sess.run(fetches = z,feed_dict={x:"hello",y:"tensorflow 1.0"}))

1.13.1
b'hello tensorflow 1.0'


#### TF2 中使用兼容版实现静态计算图

tensorFlow2.0为了确保对老版本tensorflow项目的兼容性，在tf.compat.v1子模块中保留了对TensorFlow1.0那种静态计算图构建风格的支持。
但是不同推荐这样使用。

In [3]:
import tensorflow as tf

print(tf.__version__)

g = tf.compat.v1.Graph()
# 第一步： 定义计算图
with g.as_default():
    #placeholder为占位符，执行会话时候指定填充对象
    x = tf.compat.v1.placeholder(name="x",shape=[],dtype=tf.string)
    y = tf.compat.v1.placeholder(name="y",shape=[],dtype=tf.string)
    z = tf.compat.v1.string_join([x,y],name="join",separator=" ")

# 第二步在会话中执行计算图
with tf.compat.v1.Session(graph=g) as sess:
    print(sess.run(fetches = z,feed_dict={x:"hello",y:"tensorflow v2.0 compat 1.0"}))


2.1.0
b'hello tensorflow v2.0 compat 1.0'


### 动态计算图

在TensorFlow2.0中，使用的是动态计算图和Autograph.

动态计算图已经不区分计算图的定义和执行了，而是定义后立即执行。因此称之为 Eager Excution.

> 关于 [tf.strings](https://tensorflow.google.cn/api_docs/python/tf/strings):
- 主要是关于string张量的基础操作




In [5]:
# 动态计算图在每个算子处都进行构建，构建后立即执行

x = tf.constant("hello")
y = tf.constant("TF2.0")

z = tf.strings.join([x,y],separator=' ')
print(z)

tf.Tensor(b'hello TF2.0', shape=(), dtype=string)


### TensorFlow2.0的Autograph

动态计算图运行效率相对较低。

可以用@tf.function装饰器将普通Python函数转换成和TensorFlow1.0对应的静态计算图构建代码。不需要使用会话了，一些都像原始的Python语法一样自然。

所以项目中我们使用的方式是这样子的：



- 一般会先用动态计算图调试代码，
- 然后在需要提高性能的的地方利用@tf.function切换成Autograph获得更高的效率。

当然，@tf.function的使用需要遵循一定的规范

In [13]:
# 使用autograph构建静态图

import tensorflow as tf

@tf.function   # 添加函数装饰器
def strjoin(x,y):
    z = tf.strings.join([x,y],separator=" ")
    tf.print(z)
    return z

result = strjoin(tf.constant("hello"),tf.constant("tf2"))

print(result)


hello tf2
tf.Tensor(b'hello tf2', shape=(), dtype=string)


In [14]:
import datetime

# 创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './data/autograph/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

#开启autograph跟踪
tf.summary.trace_on(graph=True, profiler=True) 

#执行autograph
result = strjoin("hello","world")

#将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name="autograph",
        step=0,
        profiler_outdir=logdir)

hello world


ProfilerNotRunningError: Cannot stop profiling. No profiler is running.

In [15]:
#启动 tensorboard在jupyter中的魔法命令
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
#启动tensorboard
%tensorboard --logdir ./data/autograph

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 30308.